# Canned-response Chatbot

This notebook creates, trains, and initializes a chatbot returns canned responses to the user.  
The model uses the DNN class from tflearn and parses input into a bag-of-words.  

The model is then implemented in a function which replicates a simple customer-service chatbot at a bank.  

More information and another example can be found in [this blogpost](https://chatbotsmagazine.com/contextual-chat-bots-with-tensorflow-4391749d0077).  


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/Engineering/Curriculum/8th Semester/Final Year Project/cs-cannedresponse'

/content/drive/.shortcut-targets-by-id/1vwz4FbZo5_X_TYcyLEdbNboj3NxTYUDp/cs-cannedresponse


In [3]:
!ls

checkpoint	    model.tflearn.data-00000-of-00001  requirements.txt
cs_cannedbot.ipynb  model.tflearn.index		       user_contacts.json
cs_prompts.json     model.tflearn.meta
data.pickle	    README.md


### Import statements

In [4]:
!pip install tflearn 
import nltk, tflearn, pickle, json, re, string, spacy, warnings
warnings.filterwarnings("ignore")
from os import path, name, system
from nltk.stem.lancaster import LancasterStemmer
from random import choice, randint, uniform
import numpy as np 
import tensorflow as tf
# Required extra installs
!python -m spacy download en_core_web_sm
!conda install scipy h5py
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 8.7 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=88ec6f8a2f637afeb0725e2f4e95a1c2a0189a450b6d7e64b6faf15e2b3a7740
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Instructions for updating:
non-resource variables are not supported in the long term
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
/bin/bash: conda: command not found


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Training data

Opens the training file and stores in a dictionary. This dictionary will be referenced throughout the notebook.  

In [5]:
with open('cs_prompts.json') as file:
    data = json.load(file)

Preprocesses and formats the training data. It builds a bag of words for each point in the training data.

In [6]:
def preprocess_data(data):
    stemmer = LancasterStemmer()

    words = []
    labels = list(data.keys())
    docs_x = []
    docs_y = []

    for label in labels:
        for pattern in data[label]['patterns']:
            tokens = nltk.word_tokenize(pattern)
            words.extend(tokens)
            docs_x.append(tokens)
            docs_y.append(label)

    # Pass over punctuation tokens
    ignored_tokens = [',', '.', '?', '!']
    words = [stemmer.stem(w.lower()) for w in words if w not in ignored_tokens]

    words = sorted(set(words))
    labels = sorted(labels)

    training = []
    output = []

    # Template for the BOW
    out_empty = list(np.zeros(len(labels)))

    for x, doc in enumerate(docs_x):
        bag = []
        stemmed = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in stemmed:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)  

    training = np.array(training)
    output = np.array(output)    
    
    return words, labels, training, output

The processed data can be stored in a pickle file for later use. If a pickle file of training data already exists, it will be read and used.

In [8]:
if path.exists('./data.pickle'):
    # If a pickle file of the processed training data exists, then it will be loaded
    with open('./data.pickle', 'rb') as file:
        words, labels, training, output = pickle.load(file)
else:
    # If no pickle file exists, the training data will be processed and saved in a pickle file
    words, labels, training, output = preprocess_data(data)
    with open('data.pickle', 'wb') as file:
        pickle.dump((words, labels, training, output), file)

### DNN model
The DNN is created using TFlearn, which is built on TensorFlow.

In [ ]:

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU') 
print(physical_devices[0])

In [ ]:
# This cell may or may not be required depending on your environment
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# Run this cell to create and train a new model

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')


#### Referencing a pretrained model
If a model has already been created and saved in a file, the following cell will pull the model into an object. This allows you to avoid constantly retraining the DNN every time the notebook is opened.

In [9]:
# Run this cell to load a previously trained model

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.load('model.tflearn')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from /content/drive/.shortcut-targets-by-id/1vwz4FbZo5_X_TYcyLEdbNboj3NxTYUDp/cs-cannedresponse/model.tflearn


#### Supporting functions

In [10]:
def clear(): 
    # Uses os.system and os.name
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
    # for mac/linux 
    else: 
        _ = system('clear') 

def bag_of_words(s, words, stemmer):
    # Creates a bag of words from a given sequence of tokens
    bag = list(np.zeros(len(words)))
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i]=1
    return np.array(bag)


### The Chat Function
The model is implemented within a function. It frames a system of rules to interpret the intent of the user and to determine a response.

In [11]:
def chat():
    with open("user_contacts.json") as h_file:
        user_contacts = json.load(h_file)
    CHAT_ENDED = False
    USER_NAME = '' 
    USER_PHONE = ''
    greeting = choice(data['greeting']['responses'])
    stemmer = LancasterStemmer()

    # Named Entity Recogntion
    ner = spacy.load('en_core_web_sm')

    def filter_punctuation(s):
        # Uses regular expressions to filter non-alphabetical characters from strings
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        return regex.sub('', s)

    def classify(user_input):
        results = model.predict([bag_of_words(user_input, words, stemmer)])
        prediction = labels[np.argmax(results)]
        response = choice(data[prediction]['responses'])
        return prediction, response

    def end_chat(inp):
        if inp.lower() in ['end', 'quit', 'stop']:
            CHAT_ENDED = True
            return True
        return False

    def get_confirmation(inp):
        prediction, response = classify(inp.lower())
        print(f"Bot: {response}")
        if prediction == 'confirm':
            return True
        else:
            return False  
        
    def get_information(check_pos, check_info, user_name):
        print(f"Bot: Please give me your {check_info}.")
        inp = input("You:")
        if end_chat(inp):
            return None
        else:
            parts = ner(inp)
            helper = []
            for part in parts:
                if part.pos_ == check_pos:
                    helper.append(part.text)
            user_info = ' '.join(helper)
            print(f"Bot: Your {check_info} is {user_info}. Is that correct?")
            confirmation_inp = filter_punctuation(input("You:").lower())

            if end_chat(confirmation_inp):
                return None

            confirmed = get_confirmation(confirmation_inp)

            if not confirmed:
                return None
        
        if check_info == 'first and last name':
            # Make a new entry in user_contact
            if user_info != '' and user_info not in user_contacts.keys():
                user_contacts[user_info] = {'phone':'','requests':[]}
        else:
            user_contacts[user_name] = {'phone':user_info,'requests':[]}
        return user_info
            
            
    clear()

    print(f"Bot: {greeting}")

    while not CHAT_ENDED:
        prediction = None
        inp = filter_punctuation(input("You:").lower())
        if end_chat(inp):
            break

        prediction, response = classify(inp)
        
        if inp == '':
            print("Bot: I'm sorry, I didn't quite get that.")
            
        elif prediction == 'deny':
            # Special case for when the bot has asked if the user needs more help
            response = choice(data['goodbye']['responses'])
            
        else:
            print(f"Bot: {response}")

            if prediction in ['open_account', 'close_account']:
                confirmation_inp = filter_punctuation(input("You:").lower())
                if end_chat(confirmation_inp):
                    break

                confirmed = get_confirmation(confirmation_inp)

                if confirmed:
                    # User wants to manage their account
                    print(f"Bot: {choice(data[prediction]['confirmed'])}")
                    # NER looks for words that begin with capital letters
                    USER_NAME = get_information('PROPN', 'first and last name', USER_NAME)
                    if USER_NAME is not None:
                        USER_PHONE = get_information('NUM', 'phone number', USER_NAME)
                        if USER_PHONE is not None:
                            if prediction in ['open_account', 'close_account']:
                                message = f"Bot: A request has been logged for {USER_NAME} to '{prediction.replace('_', ' ')}'."
                                user_contacts[USER_NAME]['requests'].append(prediction)
                            print(message)

                    USER_NAME = ''
                    USER_PHONE = ''

            elif prediction == 'goodbye' or prediction == 'deny':
                break
        
        if not CHAT_ENDED:
            print("Bot: What else I can help with?")

    with open("user_contacts.json", "w") as h_file:
        json.dump(user_contacts, h_file)

In [12]:
chat()

Bot: Hello. Welcome to FitBit CustCare. How can I help?
You:How do I set a bedtime reminder in the Fitbit app?
Bot: What else I can help with?
You:How do I set or change my sleep goal in the Fitbit app?
Bot: Hi there, how can I help?
Bot: What else I can help with?
You:Hello
Bot: Hello. Welcome to FitBit CustCare. How can I help?
Bot: What else I can help with?
You:My FitBit can’t pick up my snores
Bot: What else I can help with?


KeyboardInterrupt: ignored

# Example chat log

Bot: Hi there, how can I help?  
You: Open a new account  
Bot: You are trying to open an account. Is that correct?  
You: Yes  
Bot: Thanks for the confirmation.  
Bot: You will need to speak with a representative, but I can log a request to speed up the process.  
Bot: Please give me your first and last name.  
You: My name is Lester Morrison  
Bot: Your first and last name is Lester Morrison. Is that correct?  
You: Correct  
Bot: Thank you.  
Bot: Please give me your phone number.  
You: 3847394742  
Bot: Your phone number is 3847394742. Is that correct?  
You: Yes  
Bot: Thanks.  
Bot: A request has been logged for Lester Morrison to 'open account'  
Bot: Is there anything else I can help with?  
You: What are the hours of operation?  
Bot: All BotBank locations are open 7am-4pm Monday-Friday!  
Bot: Is there anything else I can help with?  
You: No that's all  
Bot: Glad I could help. Thanks for choosing BotBank Have a nice day.  